In [1]:
! pip install moviepy --user

In [2]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pytube import YouTube
import http.client
# from moviepy.editor import *

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [3]:
import os
import time 
import socket

from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError 
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

In [13]:
#scroll_down() 함수

def scroll_down():
    scroll_count = 0
    print("[scroll_down(): 스크롤 함수 다운 시작]") 

    last_height = wd.execute_script("return document.documentElement.scrollHeight")

    while True:
        print(f"[스크롤다운: {scroll_count}]")
     
        wd.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
        
        scroll_count += 1

        time.sleep(2)
          
        new_height = wd.execute_script("return document.documentElement.scrollHeight")
        print("0")
        if new_height == last_height:
            break 
        last_height = new_height

In [5]:
def save(url, dir_name, index, query):
    try:
        yt = YouTube(url)
        
        #영상의 제목 가져오기
        str = yt.title

        file_name = str.replace(" ", "_")
        print(file_name)

        
        #비디오 다운로드
        #progressive로 하면 해상도 최대가 720p, 대신 영상과 오디오가 합쳐진 상태
        #adaptive로 하면 해상도에 제한이 없지만, 영상과 오디오가 분리된 것을 다운받을 수 있음. moviepy로 합치기 가능.
        yt.streams.filter(progressive=True, file_extension='mp4', 
                          mime_type="video/mp4").order_by('resolution').desc().first().download(f'{query}/video/', f'{query}_{index+1}.mp4')       
        print(f"{index+1}번째 영상 : {file_name}.mp4 다운로드 완료")
        
        #오디오 다운로드
        yt.streams.filter(adaptive=True, file_extension='mp4', mime_type="audio/mp4", 
                          only_audio = True).order_by('abr').desc().first().download(f'{query}/audio/', f'{query}_{index+1}.mp4')
        
        print(f"{index+1}번째 음성 : {file_name}.mp4 다운로드 완료")
        
        os.system("ffmpeg -i {} -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav".format(f'/Users/kimminyoung/Desktop/crawling/{query}/audio/{query}_{index+1}.mp4', 
                                                                      f'/Users/kimminyoung/Desktop/crawling/{query}/audio/',
                                                                      f'{query}_{index+1}'))
        print(f"{index+1}번째 음성 : {file_name}.mp4 -> wav로 변환 완료")
        
        os.remove(f'{query}/audio/{query}_{index+1}.mp4')
        print(f"{file_name}.mp4 제거 완료")
    
        
    except KeyError as e:
        print(e)
        pass
    except (http.client.IncompleteRead) as e:
        print(e)
        pass
           
    

In [6]:
#썸네일을 통해 동영상 url저장
def click_and_save(dir_name, video, index, query):
    global scraped_count

    try:        
        time.sleep(0.5)
        
# url = video.find_element(By.XPATH, '//*[@id="thumbnail"]').get_attribute('href')
# 이상한 거 한 개만 계속 가져옴
# .get_attribute('href') 안 먹음

        url = video.find_element(By.CSS_SELECTOR, '#thumbnail').get_attribute('href')
        #url = wd.find_element(By.CSS_SELECTOR, '#thumbnail').get_attribute('href')
        
        #url_list.append(url)
        
        save(url, dir_name, index, query)
  
        scraped_count += 1
        
        
    except HTTPError as e:
        print(e)
        pass

In [15]:
#스크래핑
def scraping(dir_name, query):
    
    try:
        
        global scraped_count 

        url = f"https://www.youtube.com/c/{query}/videos"

        wd.get(url)
        wd.maximize_window()

        scroll_down()
        

        time.sleep(1)

    #     div = wd.find_element(By.XPATH,'//*[@id="items"]')
        video_list = wd.find_elements(By.XPATH,'//*[@id="items"]/ytd-grid-video-renderer')
    #     div = wd.find_element(By.XPATH,'//*[@id="contents"]/ytd-item-section-renderer')

    #     //*[@id="contents"]/ytd-item-section-renderer

    #     video_list = div.find_elements(By.CSS_SELECTOR,'#items > ytd-grid-video-renderer')
    
    except ZeroDivisionError as e:
            print(e)
            pass
    
    for index, video in enumerate(video_list):
        try:
            click_and_save(dir_name, video, index, query)
        except ElementClickInterceptedException as e: #클릭을 하는데문제가생긴경우
            print(e)
            wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
            time.sleep(1)

            click_and_save(dir_name, video, index, query)            
        except NoSuchElementException as e:
            print(e)

            wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
            time.sleep(1)

            click_and_save(dir_name, video, index, query)            
        except ConnectionResetError as e:
            print(e)
            pass
        except URLError as e:
            print(e)
            pass
        except socket.timeout as e:
            print(e)
            pass
        except socket.gaierror as e:
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            break
    try:
        print("[스크래핑 종료 (성공률: %.2ff%%)]" %(scraped_count / len(video_list) * 100.0))
    except ZeroDivisionError as e:
        print(e)
        pass
    
#     print(url_list)
    
#     save(url_list)
    
    wd.close()

In [12]:
socket.setdefaulttimeout(100) #너무 오래 걸릴 경우 끊어야하므로 최대 30초로 지정

wd = webdriver.Chrome('chromedriver')

scraped_count = 0 
path = "./" 
query = input("검색어 입력: ")

dir_name = path + query 
os.makedirs(dir_name)
print(f"[{dir_name}디렉토리 생성]") 

scraping(dir_name, query)


검색어 입력: UC5ST9VRPtnJGXOkz_agtbfg
[./UC5ST9VRPtnJGXOkz_agtbfg디렉토리 생성]
[scroll_down(): 스크롤 함수 다운 시작]
[스크롤다운: 0]
division by zero


In [9]:
# #채널 전체 다운(division by zero 나올 경우)
# #이름명으로 받을 수 없어 사용 불가 왜냐면 필요 없는 데이터가 너무 많을 수 있는 채널도 있으므로 분류를 일일이 할 수 없음
# from pytube import Channel

# c = Channel('https://www.youtube.com/channel/UCBKeJTWDZoWye2jHQq0cqyQ/videos')

# print(f'Downloading videos by: {c.channel_name}')

# for video in c.videos:
#     cnt = 0
#     video.streams.filter(progressive=True, file_extension='mp4', 
#                           mime_type="video/mp4").order_by('resolution').desc().first().download('video/', f'{video}.mp4')
    
#     cnt += 1
